In [4]:
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()


# Lecture 16: More on functional programming

## Review questions
A couple of things that came up in OH and on Canvas:

### `while` vs. `for`
There were some questions in OH about `while` versus `for`. They are almost the same:
```{r}
for (i in 1:n) {
    <body>
}
```
is the same as 
```{r}
i = 1
while (i <= n) {
    <body>
    i = i + 1
}
```

### Anonymous functions
An anonymous function is (literally) a function with no name. Some of the test cases on PS7 use this, e.g.:
```{r}
stopifnot(near(int01(function(x) x), 0.5))
#                    ^^^^^^^^^^^^^
#                 anonymous function
```
You should generally only use these for small, simple functions (max 1 line, no `{}` braces).

## The `walk` function
The `walk(v, f)` function is like `map()` in that it repeatedly calls `f` for each element of the sequence `v`. However, instead of returning the vector of outputs it returns the first argument, `v`. This is useful when you only care about a side-effect of `f`, and want to use it in a pipeline.

### Example
Use `walk` to print a summary of each column of `mpg`:

In [103]:
mpg %>% keep(is_numeric) %>% walk(~ print(quantile(.)))

Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”Warning message:
“Deprecated”

  0%  25%  50%  75% 100% 
 1.6  2.4  3.3  4.6  7.0 
    0%    25%    50%    75%   100% 
1999.0 1999.0 2003.5 2008.0 2008.0 
  0%  25%  50%  75% 100% 
   4    4    6    8    8 
  0%  25%  50%  75% 100% 
   9   14   17   19   35 
  0%  25%  50%  75% 100% 
  12   18   24   27   44 


## Iterating over multiple sequences at once
Sometimes we want to iterate over multiple sequences. For example, suppose we had a vector `mu` of means and an equal length vector `sigma` of standard deviations. For each pair `mu[[i]],sigma[[i]]` we would like to generate a five standard normal random variable using `rnorm`.

Using `map`, we could accomplish this by

In [192]:
mu = list(5, 10, -3)
sigma = list(1, 5, 10)
seq_along(mu) %>% 
  map(~rnorm(5, mu[[.]], sigma[[.]])) %>% 
  str()

List of 3
 $ : num [1:5] 5.81 3.03 5.24 3.33 5.01
 $ : num [1:5] 9.92 7.6 10.6 12.18 11.31
 $ : num [1:5] -1.65 2.76 -9.31 -2.73 -2.87


Because we don't yet know how to `map` over more than one sequence at a time, we are forced to "hack it" by iterating over `seq_along(mu)`. This hides the true intent of what we set out to accomplish, which is to perform a map over *pairs* of `(mu[i], sigma[i])`.

To iterate over two sequences at once we have the `map2` command:
```{r}
map2(seq1, seq2, f, ...)
```
will call `f(seq1[[i]], seq2[[i]], ...)` for each value of `i`. Indeed, `map2` is equivalent to:
```{r}
map2 <- function(x, y, f, ...) {
  out <- vector("list", length(x))
  for (i in seq_along(x)) {
    out[[i]] <- f(x[[i]], y[[i]], ...)
  }
  out
}
```

`map2` lets us succinctly rewrite the sampling code given above:

In [196]:
map2(mu, sigma, rnorm, n = 5)

[[1]]
[1] 5.447983 5.446291 5.066780 5.075793 4.059494

[[2]]
[1]  3.518528 19.366107  7.197257  5.081700 20.487066

[[3]]
[1] -9.655409  8.922227  4.592133  4.237675 -1.451280


We can map over arbitrarily many sequences using `pmap`. The first argument of `pmap` is a list of sequences, and the second is a function:

In [201]:
pmap(list(mu, sigma), rnorm, n = 5)

[[1]]
[1] 5.404549 5.931190 5.489289 6.243161 4.969922

[[2]]
[1] 11.36999 10.78532 16.00125 16.17708 12.38797

[[3]]
[1] 11.84802  2.68887 13.69661 11.51808 11.91021


```{r}
pmap(list(mu, sigma), rnorm, n = 5)
```
will call `rnorm(mu[[i]], sigma[[i]], n=5)`. This relies on the correct ordering of the `mu` and `sigma` options to `rnorm`. To prevent errors, you can name each sequence in the first argument:
```{r}
pmap(list(mu=mu, sigma=sigma, n=5), rnorm)
```
will call `rnorm(mu=mu[[i]], sigma=sigma[[i]], n=5)` using named parameters. This is a bit safer so I recommend using this form.

## Predicates
Predicates are functions that allow you to filter out elements of sequences based on a condition. The `keep(f)` command will return a new sequence consisting of each element where `f` evaluates to `TRUE`:

In [95]:
mpg %>% keep(is_integer) %>% print

# A tibble: 234 x 4
    year   cyl   cty   hwy
   <int> <int> <int> <int>
 1  1999     4    18    29
 2  1999     4    21    29
 3  2008     4    20    31
 4  2008     4    21    30
 5  1999     6    16    26
 6  1999     6    18    26
 7  2008     6    18    27
 8  1999     4    18    26
 9  1999     4    16    25
10  2008     4    20    28
# ... with 224 more rows


`discard()` does the opposite of `keep()`:

In [37]:
mpg %>% discard(is_integer) %>% print

# A tibble: 234 x 7
   manufacturer      model displ      trans   drv    fl   class
          <chr>      <chr> <dbl>      <chr> <chr> <chr>   <chr>
 1         audi         a4   1.8   auto(l5)     f     p compact
 2         audi         a4   1.8 manual(m5)     f     p compact
 3         audi         a4   2.0 manual(m6)     f     p compact
 4         audi         a4   2.0   auto(av)     f     p compact
 5         audi         a4   2.8   auto(l5)     f     p compact
 6         audi         a4   2.8 manual(m5)     f     p compact
 7         audi         a4   3.1   auto(av)     f     p compact
 8         audi a4 quattro   1.8 manual(m5)     4     p compact
 9         audi a4 quattro   1.8   auto(l5)     4     p compact
10         audi a4 quattro   2.0 manual(m6)     4     p compact
# ... with 224 more rows


Again, these examples work because *data frames/tibble are lists*.

`detect(f)` and `detect_index(f)` return the first element (or its index) where `f` evaluates to `TRUE`:

In [54]:
rnorm(1000) %>% detect(~ . > 2)

[1] 2.329436

In [27]:
rnorm(1000) %>% detect_index(~ abs(.) > 2)

[1] 20

## The `reduce` function
The last bit of functional programminng we will study is `reduce()`. The reduce function takes a “binary” function (i.e. a function with two primary inputs), and applies it repeatedly to a list until there is only a single element left. This takes a little bit of getting used to, but ends up being very powerful.

Let's take a simple example: summing up numbers. If I wanted to add the numbers 1 through 10 I could of course type `sum(1:10)` and get `55`. Suppose I did not know about sum. I could also write:

```{r}
> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10
[1] 55
```

We can rewrite this sum as 
```{r}
> 1 + (2 + (3 + (4 + (5 + (6 + (7 + (8 + (9 + 10))))))))
[1] 55
```
Now the sum is in the form 

$$f(1, f(2, f(3, f(4, f(5, f(6, f(7, f(8, f(9, 10)))))))))$$ 

for $f(x,y) = x + y$.

Obviously this is a silly thing to do for computing a simple sum, but the pattern of repeatedly applying a binary function $f(x,y)$, where $y$ is the output from a previous application of the function, actually turns up a lot in programming. The `reduce` function accomplishes this for us:

In [75]:
reduce(1:10, `+`)

[1] 55

## `accumulate`
A closely related variant of `reduce` is `accumulate`. In the above example, instead of returning 

$$f(1, f(2, f(3, f(4, f(5, f(6, f(7, f(8, f(9, 10))))))))),$$ 

accumulate would return a *vector* containing
all of the partially evaluated sums:

$$\texttt{accumulate(f, 1:10)} = (f(1, 2), f(3, f(1, 2)), f(4, f(3, f(1, 2))), \dots)$$

In [21]:
1:10 %>% accumulate(`+`)

 [1]  1  3  6 10 15 21 28 36 45 55

### Example
Suppose we have a list of vectors $(v_1,\dots,v_n)$ and we want to find their intersection. Mathematically this equals $$I_n:=\bigcap_{i=1}^n v_n=v_n\cap I_{n-1},$$ so we can use reduce to compute it:

In [77]:
vs = list(
  c(1, 3, 5, 6, 10),
  c(1, 2, 3, 7, 8, 10),
  c(1, 2, 3, 4, 8, 9, 10)
)
vs %>% reduce(intersect)

[1]  1  3 10

### Example
Let's say we want to concatenate together a bunch of strings, and because we haven't been coming to lecture we don't know about `str_c`. In fact, we only know about the following function which concatenates just two strings at a time:

In [14]:
cat2 = function(x, y, ...) {
    paste0(x, y, ...)
}

Use `cat2` and `reduce` to build a function which will concatenate whole vectors of strings together.

Let's think about what our new function will do. If we pass it the vector `c('a','b','c','d')` we should get back the string `'abcd'`. We could accomplish this using `cat2`:

In [15]:
v = c('a', 'b', 'c', 'd')
cat2(cat2(cat2(v[1], v[2]), v[3]), v[4])

[1] "abcd"

This pattern suggests how we might use `reduce`:

In [18]:
v %>% reduce(cat2)

[1] "abcd"

### Example
Given a vector $v$, the *cumulative product* of $v$ is a vector $p$ such that $$p_j = \prod_{i=1}^j v_i.$$
For example, the cumulative product of $(1,2,3,4)$ is $(1,2,6,24)$ (i.e. the factorials). 

Use `accumulate` to write a function `cumprod(v)` that computes the cumulative product.

In [24]:
# cumprod(v)

## Closures
When we create a function
```{r}
f = function(x) {
    <do somethings with x>
}
```
that function has access to:
- all of the variables defined inside the function (its arguments plus whatever other variables we create in the body of the function), *as well as*
- all of the variables defined "outside" the function.

Example:

In [29]:
y = 1
f = function(x) {
    x + y
}

Now `f` is a function that will add one to its argument:

In [30]:
f(1)

[1] 2

In fact, we can change the behavior of `f` by changing `y`. Observe:

In [34]:
y = 2
f(1)  # now f adds 2 to x

[1] 3

In [35]:
y = NA
f(1) 

[1] NA

In this example `f` is what's called a *closure*, because it *encloses* all the variables defined outside of `f`, as well as those defined inside of it.

Closures become useful when we think about creating functions inside of other functions:

In [47]:
power = function(exponent) {
  function(x) {
    x ^ exponent
  }
}
square = power(2)
cube = power(3)

What is `square` now? It's a function:

In [49]:
square(3)
cube(4)

[1] 9

[1] 64

As we said, `square` is a closure. The list of variables it encloses is called its *environment*. You can look at this list by using the command of that name:

In [50]:
square
as.list(environment(square))

function(x) {
    x ^ exponent
  }
<environment: 0x7fe8b18b6958>

$exponent
[1] 2


In [53]:
as.list(environment(cube))

$exponent
[1] 3


### Memoization
A good use for closures is something called *memoization*. Memoization lets you save the results of a function call for future use. This is very useful in combination with recursion.

To implement memoization, we define a list in the enclosing enviroment of the function we wish to memoize. Then, each time we call the function, we check to see if the value is already saved in the list. If so, we return it; otherwise, we do the computation and store it for future use.

Consider the following recursive function:

In [54]:
f = function(k) {
    if (k <= 2) {
        1
    } else {
        k^2 * f(k - 1) - k * f(k - 2)
    }
}

This function is slow to compute for large $k$:

In [56]:
library(microbenchmark)
microbenchmark(
    f(10),
    f(20)
) %>% print

Unit: microseconds
  expr      min       lq       mean   median       uq       max neval
 f(10)   40.272   42.286   47.87195   44.059   48.437   111.632   100
 f(20) 5050.245 5298.817 5997.62755 5566.617 6409.959 16303.364   100


To speed up `f` we will memoize it:

In [83]:
memo = rep(NA_real_, 100)
fmemo = function(k) {
    if (is.na(memo[[k]])) {
        ret = NA
        if (k <= 2) {
            ret = 1
        } else {
            ret = k^2 * fmemo(k - 1) - k * fmemo(k - 2)
        }
        memo[[k]] <<- ret
    }
    memo[[k]]
}

Note the use of the `<<-` operator. This is required in order to make assignments to the enclosed variable `memo`.

In [84]:
library(microbenchmark)
microbenchmark(
    f(10),
    f(20),
    fmemo(10),
    fmemo(20)
) %>% print

Unit: nanoseconds
      expr     min        lq       mean  median        uq     max neval
     f(10)   40148   42348.5   45627.00   43727   46998.0   59960   100
     f(20) 5061218 5196638.0 5657780.17 5419740 5844943.0 7511087   100
 fmemo(10)     532     744.0   85509.24     923    2988.5 8335469   100
 fmemo(20)     531     702.5    2194.34     918    2534.0   27192   100


## Error handling
All of the usages of `map` and `reduce` so far have been toy examples with no chance of failure. In real data you will encounter errors. If you do not handle them then your computation will return errors or fail:

In [10]:
# error
# map_dbl(list("a", -1, 2, 3, 4), log)

To handle this type of situation, `tidyverse` provides you with the `safely()` command. `safely()` is an adverb: it takes a function (a verb) and returns a modified version. The modified function will never throw an error; instead, it returns a list with two elements:
1. `result` is the original result. If there was an error, this will be `NULL`.
2. `error` is an error object. If the operation was successful, this will be `NULL`.



In [183]:
(res = map(list("a", -1, 2, 3, 4), safely(log))) %>% str

Warning message in .f(...):
“NaNs produced”

List of 5
 $ :List of 2
  ..$ result: NULL
  ..$ error :List of 2
  .. ..$ message: chr "non-numeric argument to mathematical function"
  .. ..$ call   : language log(x = x, base = base)
  .. ..- attr(*, "class")= chr [1:3] "simpleError" "error" "condition"
 $ :List of 2
  ..$ result: num NaN
  ..$ error : NULL
 $ :List of 2
  ..$ result: num 0.693
  ..$ error : NULL
 $ :List of 2
  ..$ result: num 1.1
  ..$ error : NULL
 $ :List of 2
  ..$ result: num 1.39
  ..$ error : NULL


To get the results we need to extract the `result` attribute from each element of `res`. We already learned how to do this using `map`:

In [182]:
map(res, "result") %>% str

List of 5
 $ : NULL
 $ : num NaN
 $ : num 0.693
 $ : num 1.1
 $ : num 1.39


Alternatively, thinking of `res` as a matrix with two columns, we can transpose `res` and take the `result` attribute:

In [181]:
transpose(res)$result %>% str

List of 5
 $ : NULL
 $ : num NaN
 $ : num 0.693
 $ : num 1.1
 $ : num 1.39


The related command `possibly` will return a default value wherever an error is encountered:

In [180]:
map(list('a', -1, 2, 3), possibly(log, NA_real_)) %>% str

Warning message in .f(...):
“NaNs produced”

List of 4
 $ : num NA
 $ : num NaN
 $ : num 0.693
 $ : num 1.1


Finally, to capture and suppress the warning message we can use `quietly()`:

In [188]:
map(list(-1, 2, 3), quietly(log)) %>% str

List of 3
 $ :List of 4
  ..$ result  : num NaN
  ..$ output  : chr ""
  ..$ warnings: chr "NaNs produced"
  ..$ messages: chr(0) 
 $ :List of 4
  ..$ result  : num 0.693
  ..$ output  : chr ""
  ..$ warnings: chr(0) 
  ..$ messages: chr(0) 
 $ :List of 4
  ..$ result  : num 1.1
  ..$ output  : chr ""
  ..$ warnings: chr(0) 
  ..$ messages: chr(0) 
